# Processamento Estatístico de Sinais - TI 0124 - Trabalho computacional 1


## Prof. Charles Casimiro Cavalcante, Dr.
## Wireless Telecommunications Research Group - GTEL
## Department of Teleinformatics Engineering - DETI
## Federal University of Ceará (UFC) - Brazil
## URL: [charlescasimiro.github.io](https://charlescasimiro.github.io/)

## Alunos:

- Kelvin Leandro Martins - 540006
- Paulo Ricardo Menezes Soares - 537440
- Pedro Leinos Falcão Cunha - 542114


# Importando bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import FloatSlider, IntSlider

# 1. implementar um processo estocástico AR (autor regressivo) de ordem M - tornar a ordem M uma escolha do usuário assim como os coeficientes do modelo AR;

## Modelo AR:
## $x(n) = b_1x(n-1) + b_2x(n-2) + ... + b_Mx(n-M) + v(n)$

### onde $b_k = -a_k$ e $v(n)$ é um processo de ruído branco

In [2]:
# Variáveis globais para manter o estado dos coeficientes
global_coeficientes_AR = {}

def gerar_processo_AR(coeficientes, ordem_M, amostras, seed):
    rng = np.random.RandomState(seed)

    # Inicializa o processo AR com zeros
    serie_AR = np.zeros(amostras)

    # Gera ruído branco
    ruido_branco = rng.normal(0, 1, amostras)

    # Construção do processo AR
    for t in range(ordem_M, amostras):
        serie_AR[t] = np.dot(coeficientes, serie_AR[t-ordem_M:t][::-1]) + ruido_branco[t]

    return serie_AR

def plot_serie_AR(serie_AR, ordem_M):
    # Plota a série gerada
    plt.figure(figsize=(10, 5))
    plt.plot(serie_AR, label=f"Processo AR({ordem_M})")
    plt.title(f"Série Temporal Gerada - AR({ordem_M})")
    plt.xlabel("Tempo")
    plt.ylabel("Valor")
    plt.legend()
    plt.grid()
    plt.show()

@interact(
    ordem_M=widgets.IntSlider(min=1, max=10, step=1, value=5),
    amostras=widgets.IntSlider(min=100, max=5000, step=100, value=2500),
    seed=widgets.IntSlider(min=0, max=100, step=1, value=50)
)
def main_AR(ordem_M, amostras, seed):
    global global_coeficientes_AR

    # Atualiza os coeficientes globais para manter estado
    if len(global_coeficientes_AR) < ordem_M:
        for i in range(len(global_coeficientes_AR), ordem_M):
            global_coeficientes_AR[f"a{i+1}"] = 0.0
    else:
        global_coeficientes_AR = {k: global_coeficientes_AR[k] for k in list(global_coeficientes_AR.keys())[:ordem_M]}

    coeficientes_widgets = {
        f"a{i+1}": widgets.FloatSlider(min=-1.0, max=1.0, step=0.05, value=global_coeficientes_AR[f"a{i+1}"])
        for i in range(ordem_M)
    }

    @interact(**coeficientes_widgets)
    def update_AR(**coeficientes):
        global global_coeficientes_AR
        # Atualiza o estado global dos coeficientes
        global_coeficientes_AR.update(coeficientes)

        # Coeficientes do modelo
        coeficientes_AR = [coeficientes[f"a{i+1}"] for i in range(ordem_M)]

        # Gera a série do processo AR
        serie_AR = gerar_processo_AR(coeficientes_AR, ordem_M, amostras, seed)

        # Plota a série do processo AR
        plot_serie_AR(serie_AR, ordem_M)

interactive(children=(IntSlider(value=5, description='ordem_M', max=10, min=1), IntSlider(value=2500, descript…

# 2. implementar um processo estocástico MA (média móvel) de ordem K - tornar a ordem K uma escolha do usuário, assim como os coeficientes do modelo MA;

## Modelo MA:
## $x(n) = v(n) + b_1 v(n-1) + b_2 v(n-2) + ... + b_K v(n-K)$

### onde $v(n)$ é um processo de ruído branco de média 0 e variância $σ_v^2$

In [8]:
global_coeficientes_MA = {}

def gerar_processo_MA(coeficientes, ordem_K, amostras, seed):
    rng = np.random.RandomState(seed)

    # Inicializar o processo MA com zeros
    serie_MA = np.zeros(amostras)

    # Gera ruído branco
    ruido_branco = rng.normal(0, 1, amostras)

    # Construção do processo MA
    for t in range(amostras):
        for k in range(ordem_K):
            if t - k >= 0:
                serie_MA[t] += coeficientes[k] * ruido_branco[t - k]

    return serie_MA

def plot_serie_MA(serie_MA, ordem_K):
    # Plota a série gerada
    plt.figure(figsize=(10, 5))
    plt.plot(serie_MA, label=f"MA({ordem_K})")
    plt.title(f"Processo MA({ordem_K})")
    plt.xlabel("Tempo")
    plt.ylabel("Valor")
    plt.legend()
    plt.grid()
    plt.show()

@interact(
    ordem_K=widgets.IntSlider(min=1, max=10, step=1, value=3),
    amostras=widgets.IntSlider(min=100, max=5000, step=100, value=2500),
    seed=widgets.IntSlider(min=0, max=100, step=1, value=50)
)
def main_MA(ordem_K, amostras, seed):
    global global_coeficientes_MA

    # Atualiza os coeficientes globais para manter estado
    if len(global_coeficientes_MA) < ordem_K:
        for i in range(len(global_coeficientes_MA), ordem_K):
            global_coeficientes_MA[f"b{i+1}"] = 0.0
    else:
        global_coeficientes_MA = {k: global_coeficientes_MA[k] for k in list(global_coeficientes_MA.keys())[:ordem_K]}

    coeficientes_widgets = {
        f"b{i+1}": widgets.FloatSlider(min=-1.0, max=1.0, step=0.05, value=global_coeficientes_MA[f"b{i+1}"])
        for i in range(ordem_K)
    }

    @interact(**coeficientes_widgets)
    def update_MA(**coeficientes):
        global global_coeficientes_MA
        # Atualiza o estado global dos coeficientes
        global_coeficientes_MA.update(coeficientes)

        # Coeficientes do modelo
        coeficientes_MA = [coeficientes[f"b{i+1}"] for i in range(ordem_K)]

        # Gera a série do processo MA
        serie_MA = gerar_processo_MA(coeficientes_MA, ordem_K, amostras, seed)

        # Plota a série do processo MA
        plot_serie_MA(serie_MA, ordem_K)


interactive(children=(IntSlider(value=3, description='ordem_K', max=10, min=1), IntSlider(value=2500, descript…

# 3. implementar um processo estocástico ARMA (auto regressivo e média móvel) de ordem (M, K) - tornar a ordem (M, K) uma escolha do usuário, assim como os coeficientes do modelo MA;

## Modelo ARMA:
## $x(n) + a_1x(n-1) + a_2x(n-2) + ... + a_Mx(n-M) = v(n) + b_1 v(n-1) + b_2 v(n-2) + ... + b_K v(n-K)$

### onde $v(n)$ é um processo de ruído branco de média 0 e variância $σ_v^2$

### onde $a_1, ..., a_M$ e $b_1, ..., b_K$ são parâmetros ARMA.

In [4]:
# Variáveis globais para manter o estado dos coeficientes
global_coeficientes_ARMA_AR = {}
global_coeficientes_ARMA_MA = {}

def gerar_ARMA(ordem_M, ordem_K, coeficientes_AR, coeficientes_MA, amostras, seed):
    rng = np.random.RandomState(seed)

    # Inicializar série e ruído branco
    serie_ARMA = np.zeros(amostras)
    ruido_branco = np.random.normal(0, 1, amostras)  # Ruído branco

    # Gerar a série ARMA
    for t in range(max(ordem_M, ordem_K), amostras):
        # Termo AR
        ar_term = sum(coeficientes_AR[i] * serie_ARMA[t - i - 1] for i in range(ordem_M))
        # Termo MA
        ma_term = sum(coeficientes_MA[i] * ruido_branco[t - i - 1] for i in range(ordem_K))
        # Série
        serie_ARMA[t] = ar_term + ma_term + ruido_branco[t]

    return serie_ARMA

def plotar_arma(serie_ARMA, ordem_M, ordem_K):
    plt.figure(figsize=(10, 6))
    plt.plot(serie_ARMA, label=f'ARMA({ordem_M},{ordem_K})')
    plt.title(f'Processo ARMA({ordem_M},{ordem_K})')
    plt.xlabel('Tempo')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
    plt.show()

@interact(
    ordem_M=widgets.IntSlider(min=1, max=10, step=1, value=2),
    ordem_K=widgets.IntSlider(min=1, max=10, step=1, value=2),
    amostras=widgets.IntSlider(min=100, max=5000, step=100, value=2500),
    seed=widgets.IntSlider(min=0, max=100, step=1, value=50)
)
def main_ARMA(ordem_M, ordem_K, amostras, seed):
    global global_coeficientes_ARMA_AR, global_coeficientes_ARMA_MA

    # Atualiza os coeficientes globais para AR
    if len(global_coeficientes_ARMA_AR) < ordem_M:
        for i in range(len(global_coeficientes_ARMA_AR), ordem_M):
            global_coeficientes_ARMA_AR[f"a{i+1}"] = 0.0
    else:
        global_coeficientes_ARMA_AR = {k: global_coeficientes_ARMA_AR[k] for k in list(global_coeficientes_ARMA_AR.keys())[:ordem_M]}

    # Atualiza os coeficientes globais para MA
    if len(global_coeficientes_ARMA_MA) < ordem_K:
        for i in range(len(global_coeficientes_ARMA_MA), ordem_K):
            global_coeficientes_ARMA_MA[f"b{i+1}"] = 0.0
    else:
        global_coeficientes_ARMA_MA = {k: global_coeficientes_ARMA_MA[k] for k in list(global_coeficientes_ARMA_MA.keys())[:ordem_K]}

    coeficientes_widgets_AR = {
        f"a{i+1}": widgets.FloatSlider(min=-1.0, max=1.0, step=0.05, value=global_coeficientes_ARMA_AR[f"a{i+1}"])
        for i in range(ordem_M)
    }

    coeficientes_widgets_MA = {
        f"b{i+1}": widgets.FloatSlider(min=-1.0, max=1.0, step=0.05, value=global_coeficientes_ARMA_MA[f"b{i+1}"])
        for i in range(ordem_K)
    }

    @interact(**coeficientes_widgets_AR, **coeficientes_widgets_MA)
    def update_ARMA(**coeficientes):
        global global_coeficientes_ARMA_AR, global_coeficientes_ARMA_MA

        # Atualiza o estado global dos coeficientes
        global_coeficientes_ARMA_AR.update({k: coeficientes[k] for k in coeficientes if k.startswith('a')})
        global_coeficientes_ARMA_MA.update({k: coeficientes[k] for k in coeficientes if k.startswith('b')})

        # Coeficientes do modelo ARMA
        coeficientes_AR = [global_coeficientes_ARMA_AR[f"a{i+1}"] for i in range(ordem_M)]
        coeficientes_MA = [global_coeficientes_ARMA_MA[f"b{i+1}"] for i in range(ordem_K)]

        # Gera a série do processo ARMA
        serie_ARMA = gerar_ARMA(ordem_M, ordem_K, coeficientes_AR, coeficientes_MA, amostras, seed)

        # Plota o modelo ARMA
        plotar_arma(serie_ARMA, ordem_M, ordem_K)

interactive(children=(IntSlider(value=2, description='ordem_M', max=10, min=1), IntSlider(value=2, description…